In [21]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

In [2]:
#check datatypes
def check_dtypes2(dataframe):
    test =dataframe.dtypes
    x=0
    for i in range (len(dataframe.columns)):
        print("COL NAME: ",dataframe.columns[i])
        print("  TYPE: ",test[x])
        print("  % OF UNIQUE VALUES: ", (len(pd.unique(dataframe[dataframe.columns[i]]))/len(dataframe[dataframe.columns[i]]))*100 )
        print("  Values: ",pd.unique(dataframe[dataframe.columns[i]]))
        print("  SAMPLE: ",dataframe.iloc[0][dataframe.columns[i]])
        print()
        x+=1
    return()
#dealing with null values

#null values info
def null_find(dataframe):
    null_df = pd.DataFrame(round(dataframe.isna().sum()/len(dataframe),4)*100)
    null_df = null_df.reset_index()
    null_df.columns = ['header','percentage_nulls']
    return(null_df)
           
#null values reduction
def null_reduction(dataframe,porcentage):
    null_df=null_find(dataframe)
    columns_drop = null_df[null_df['percentage_nulls']>porcentage]['header']
    return dataframe.drop(columns_drop,axis=1)

# print non zero null columns

def print_nan(dataframe):
    null_df = null_find(dataframe)
    null_df = null_df[null_df['percentage_nulls']>0]
    print(null_df)
    return()


#scaler
def scaleColumns(df, cols_to_scale):
    scaler = preprocessing.StandardScaler()

    for col in cols_to_scale:
        df[col] = pd.DataFrame(scaler.fit_transform(pd.DataFrame(df[col])),columns=[col])
    return df


Read that data into Python and call the dataframe churnData.

In [3]:
churnData = pd.read_csv('./files_for_lab/Customer-Churn.csv')
churnData.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,MonthlyCharges,TotalCharges,Churn
0,Female,0,Yes,No,1,No,No,Yes,No,No,No,No,Month-to-month,29.85,29.85,No
1,Male,0,No,No,34,Yes,Yes,No,Yes,No,No,No,One year,56.95,1889.5,No
2,Male,0,No,No,2,Yes,Yes,Yes,No,No,No,No,Month-to-month,53.85,108.15,Yes
3,Male,0,No,No,45,No,Yes,No,Yes,Yes,No,No,One year,42.30,1840.75,No
4,Female,0,No,No,2,Yes,No,No,No,No,No,No,Month-to-month,70.70,151.65,Yes


Check the datatypes of all the columns in the data. You would see that the column TotalCharges is object type. Convert this column into numeric type using pd.to_numeric function.

In [4]:
check_dtypes2(churnData)

COL NAME:  gender
  TYPE:  object
  % OF UNIQUE VALUES:  0.02839698991906858
  Values:  ['Female' 'Male']
  SAMPLE:  Female

COL NAME:  SeniorCitizen
  TYPE:  int64
  % OF UNIQUE VALUES:  0.02839698991906858
  Values:  [0 1]
  SAMPLE:  0

COL NAME:  Partner
  TYPE:  object
  % OF UNIQUE VALUES:  0.02839698991906858
  Values:  ['Yes' 'No']
  SAMPLE:  Yes

COL NAME:  Dependents
  TYPE:  object
  % OF UNIQUE VALUES:  0.02839698991906858
  Values:  ['No' 'Yes']
  SAMPLE:  No

COL NAME:  tenure
  TYPE:  int64
  % OF UNIQUE VALUES:  1.0364901320460032
  Values:  [ 1 34  2 45  8 22 10 28 62 13 16 58 49 25 69 52 71 21 12 30 47 72 17 27
  5 46 11 70 63 43 15 60 18 66  9  3 31 50 64 56  7 42 35 48 29 65 38 68
 32 55 37 36 41  6  4 33 67 23 57 61 14 20 53 40 59 24 44 19 54 51 26  0
 39]
  SAMPLE:  1

COL NAME:  PhoneService
  TYPE:  object
  % OF UNIQUE VALUES:  0.02839698991906858
  Values:  ['No' 'Yes']
  SAMPLE:  No

COL NAME:  OnlineSecurity
  TYPE:  object
  % OF UNIQUE VALUES:  0.0425954848

()

In [5]:
churnData['TotalCharges']= pd.to_numeric(churnData['TotalCharges'], errors='coerce')

In [6]:
check_dtypes2(churnData)

COL NAME:  gender
  TYPE:  object
  % OF UNIQUE VALUES:  0.02839698991906858
  Values:  ['Female' 'Male']
  SAMPLE:  Female

COL NAME:  SeniorCitizen
  TYPE:  int64
  % OF UNIQUE VALUES:  0.02839698991906858
  Values:  [0 1]
  SAMPLE:  0

COL NAME:  Partner
  TYPE:  object
  % OF UNIQUE VALUES:  0.02839698991906858
  Values:  ['Yes' 'No']
  SAMPLE:  Yes

COL NAME:  Dependents
  TYPE:  object
  % OF UNIQUE VALUES:  0.02839698991906858
  Values:  ['No' 'Yes']
  SAMPLE:  No

COL NAME:  tenure
  TYPE:  int64
  % OF UNIQUE VALUES:  1.0364901320460032
  Values:  [ 1 34  2 45  8 22 10 28 62 13 16 58 49 25 69 52 71 21 12 30 47 72 17 27
  5 46 11 70 63 43 15 60 18 66  9  3 31 50 64 56  7 42 35 48 29 65 38 68
 32 55 37 36 41  6  4 33 67 23 57 61 14 20 53 40 59 24 44 19 54 51 26  0
 39]
  SAMPLE:  1

COL NAME:  PhoneService
  TYPE:  object
  % OF UNIQUE VALUES:  0.02839698991906858
  Values:  ['No' 'Yes']
  SAMPLE:  No

COL NAME:  OnlineSecurity
  TYPE:  object
  % OF UNIQUE VALUES:  0.0425954848

()

Check for null values in the dataframe. Replace the null values.

In [7]:
null_find(churnData)

,header,percentage_nulls
0,gender,0.00
1,SeniorCitizen,0.00
2,Partner,0.00
3,Dependents,0.00
4,tenure,0.00
5,PhoneService,0.00
6,OnlineSecurity,0.00
7,OnlineBackup,0.00
8,DeviceProtection,0.00
9,TechSupport,0.00


In [8]:
churnData.dropna(subset=['TotalCharges'], inplace = True)

In [9]:
churnData.reset_index(drop=True, inplace = True)

In [10]:
null_find(churnData)

,header,percentage_nulls
0,gender,0.0
1,SeniorCitizen,0.0
2,Partner,0.0
3,Dependents,0.0
4,tenure,0.0
5,PhoneService,0.0
6,OnlineSecurity,0.0
7,OnlineBackup,0.0
8,DeviceProtection,0.0
9,TechSupport,0.0


Use the following features: tenure, SeniorCitizen, MonthlyCharges and TotalCharges:

    Scale the features either by using normalizer or a standard scaler.

In [11]:
churnData = scaleColumns(churnData, ['SeniorCitizen','MonthlyCharges','TotalCharges'])


In [12]:
churnData.SeniorCitizen

0      -0.440327
1      -0.440327
2      -0.440327
3      -0.440327
4      -0.440327
          ...   
7027   -0.440327
7028   -0.440327
7029   -0.440327
7030    2.271039
7031   -0.440327
Name: SeniorCitizen, Length: 7032, dtype: float64

Split the data into a training set and a test set.

In [18]:
Xf=churnData.drop(['Churn'], axis=1)
yf=churnData['Churn']
Xf_num=Xf.select_dtypes(include = np.number)
Xf_cat = pd.get_dummies(Xf.select_dtypes(include = 'object'), drop_first=True)
Xf=pd.concat([Xf_num, Xf_cat], axis=1) 

    
X_train, X_test, y_train, y_test = train_test_split(Xf, yf, test_size=0.3, random_state=42)






Fit a logistic regression model on the training data.

In [23]:
model = LogisticRegression()
model.fit( X_train, y_train )


LogisticRegression()

In [24]:
predicted = model.predict(X_test)

Check the accuracy on the test data.

In [28]:
print(classification_report(y_test, predicted))

              precision    recall  f1-score   support

          No       0.84      0.89      0.86      1549
         Yes       0.63      0.52      0.57       561

    accuracy                           0.79      2110
   macro avg       0.73      0.71      0.72      2110
weighted avg       0.78      0.79      0.78      2110



# Managing imbalance in the dataset

    Check for the imbalance.

In [29]:
churnData['Churn'].value_counts()

No     5163
Yes    1869
Name: Churn, dtype: int64

Use the resampling strategies used in class for upsampling and downsampling to create a balance between the two classes.

## Downsampling

In [35]:

category_0 = churnData[churnData['Churn'] == 'No']
category_1 = churnData[churnData['Churn'] == 'Yes']

# We pick a random sample of rows from of observations belonging to "category_0"
# in the same amount of observations belonging to "category_1"
category_0 = category_0.sample(len(category_1))
print(category_0.shape)
print(category_1.shape)

data = pd.concat([category_0, category_1], axis=0)
#shuffling the data
data = data.sample(frac=1)
data['Churn'].value_counts()

(1869, 16)
(1869, 16)


No     1869
Yes    1869
Name: Churn, dtype: int64

In [36]:
Xf=data.drop(['Churn'], axis=1)
yf=data['Churn']
Xf_num=Xf.select_dtypes(include = np.number)
Xf_cat = pd.get_dummies(Xf.select_dtypes(include = 'object'), drop_first=True)
Xf=pd.concat([Xf_num, Xf_cat], axis=1) 

    
X_train, X_test, y_train, y_test = train_test_split(Xf, yf, test_size=0.3, random_state=42)
model = LogisticRegression()
model.fit( X_train, y_train )
predicted = model.predict(X_test)
print(classification_report(y_test, predicted))

              precision    recall  f1-score   support

          No       0.81      0.71      0.76       585
         Yes       0.72      0.82      0.77       537

    accuracy                           0.76      1122
   macro avg       0.77      0.77      0.76      1122
weighted avg       0.77      0.76      0.76      1122



# Upsampling

In [39]:
category_0 = churnData[churnData['Churn'] == 'No']
category_1 = churnData[churnData['Churn'] == 'Yes']

# Upsampling 
# As we are going to repeat observations, the random samples can be picked more then once,
# threfore we need to use the keyword: replace=True
category_1 = category_1.sample(len(category_0), replace=True)  
print(category_1.shape)

data = pd.concat([category_0, category_1], axis=0)
#shuffling the data
data = data.sample(frac=1)
print(data['Churn'].value_counts())

(5163, 16)
No     5163
Yes    5163
Name: Churn, dtype: int64


In [40]:
Xf=data.drop(['Churn'], axis=1)
yf=data['Churn']
Xf_num=Xf.select_dtypes(include = np.number)
Xf_cat = pd.get_dummies(Xf.select_dtypes(include = 'object'), drop_first=True)
Xf=pd.concat([Xf_num, Xf_cat], axis=1) 

    
X_train, X_test, y_train, y_test = train_test_split(Xf, yf, test_size=0.3, random_state=42)
model = LogisticRegression()
model.fit( X_train, y_train )
predicted = model.predict(X_test)
print(classification_report(y_test, predicted))

              precision    recall  f1-score   support

          No       0.79      0.71      0.75      1548
         Yes       0.74      0.81      0.77      1550

    accuracy                           0.76      3098
   macro avg       0.76      0.76      0.76      3098
weighted avg       0.76      0.76      0.76      3098

